# Q3a Tagging prediction

In [1]:
import re
import jieba
import pandas as pd
import numpy as np

In [2]:
# Loading the training set

In [3]:
df=pd.read_csv("offsite-tagging-training-set.csv",index_col="id")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4027 entries, 3443 to 77253
Data columns (total 12 columns):
tags           3895 non-null object
text           3894 non-null object
Unnamed: 3     18 non-null object
Unnamed: 4     14 non-null object
Unnamed: 5     4 non-null object
Unnamed: 6     4 non-null object
Unnamed: 7     3 non-null object
Unnamed: 8     2 non-null object
Unnamed: 9     1 non-null object
Unnamed: 10    1 non-null object
Unnamed: 11    1 non-null object
Unnamed: 12    1 non-null object
dtypes: object(12)
memory usage: 409.0+ KB


In [5]:
# Remove the redundant columns
for i in range(3,13):
    df.drop(["Unnamed: "+str(i)],axis=1,inplace=True)

In [6]:
df=df.dropna()

In [7]:
# Create a collection of Stopwords
Stopword='的了和是就都而及與著或一個沒有我們你們妳們他們她們是否上下只乎些如果因為所以這麼也亦仍從此于但且'

In [8]:
# Create a column 'Clean' by removing the unnecessary wordings
df['Clean']=df['text'].str.findall('[^\da-zA-Z\s\W_%s]'%Stopword)

In [9]:
row,col=df.shape
row,col

(3894, 3)

In [10]:
df.head()

,tags,text,Clean
id,,,
3443,足球,利物浦重賽擊敗乙組仔 英足盃過關 英格蘭足總盃第三圈今晨重賽，貴為英超勁旅的利物浦上場被乙組...,"[利, 物, 浦, 重, 賽, 擊, 敗, 乙, 組, 仔, 英, 足, 盃, 過, 關, ..."
76056,足球,【中超】恒大「暴力戰」絕殺國安 楊智反重力插水惹爭議（有片） 中超首輪賽事重頭戲，廣州恒大主...,"[中, 超, 恒, 大, 暴, 力, 戰, 絕, 殺, 國, 安, 楊, 智, 反, 重, ..."
93405,足球,【歐霸決賽】阿積士控球率起腳佔優 隊長卡拉臣輸波不服氣 阿積士以歐洲主要決賽最年輕、平均22...,"[歐, 霸, 決, 賽, 阿, 積, 士, 控, 球, 率, 起, 腳, 佔, 優, 隊, ..."
26767,足球,【歐國盃】韋莫斯澄清更衣室未內訌 盼以團結力量挫愛爾蘭 今晚3場直播\n\nE組｜比利時Vs...,"[歐, 國, 盃, 韋, 莫, 斯, 澄, 清, 更, 衣, 室, 未, 內, 訌, 盼, ..."
20843,梁振英,王維基參選 點解？ 王維基在宣布有意出選的記者會上，打出ABC，Anyone But CY的...,"[王, 維, 基, 參, 選, 點, 解, 王, 維, 基, 在, 宣, 布, 意, 出, ..."


In [11]:
# Using jieba to create the text into a list of Chinese vocabulary
# This improves the text classification
for i in range(row):
    df.iloc[i,2]=''.join(df.iloc[i,2])
    df.iloc[i,2]=' '.join(jieba.cut(df.iloc[i,2],cut_all=False))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\HoT\AppData\Local\Temp\jieba.cache
Loading model cost 0.574 seconds.
Prefix dict has been built succesfully.


In [12]:
df.head()

,tags,text,Clean
id,,,
3443,足球,利物浦重賽擊敗乙組仔 英足盃過關 英格蘭足總盃第三圈今晨重賽，貴為英超勁旅的利物浦上場被乙組...,利物浦 重賽 擊敗 乙組 仔英足 盃 過關 英格蘭足 總 盃 第三圈 今晨 重賽 貴 英超 ...
76056,足球,【中超】恒大「暴力戰」絕殺國安 楊智反重力插水惹爭議（有片） 中超首輪賽事重頭戲，廣州恒大主...,中 超恒大 暴力 戰絕 殺國安 楊智反 重力 插水 惹 爭議 片中 超首 輪賽 事 重頭 戲...
93405,足球,【歐霸決賽】阿積士控球率起腳佔優 隊長卡拉臣輸波不服氣 阿積士以歐洲主要決賽最年輕、平均22...,歐霸 決賽 阿積士 控球 率起 腳 佔 優隊 長 卡拉 臣輸波 不服 氣阿積士 歐洲 主要 ...
26767,足球,【歐國盃】韋莫斯澄清更衣室未內訌 盼以團結力量挫愛爾蘭 今晚3場直播\n\nE組｜比利時Vs...,歐國 盃 韋莫斯 澄清 更衣室 未內訌 盼 團結 力量 挫愛爾蘭 今晚 場 直播 組 比利 ...
20843,梁振英,王維基參選 點解？ 王維基在宣布有意出選的記者會上，打出ABC，Anyone But CY的...,王維基 參選點 解王 維基 在 宣布 意出 選記者 會 打 出口 號說 要 香港 代做 點事...


In [13]:
# Split the train set in to a sub-train set and a sub-test set
# This can be used for estimate the performance of model

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X=df.iloc[:,2].values
y=df.iloc[:,0].values

In [16]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=21)

In [ ]:
# Feature extraction 

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

In [18]:
Count=CountVectorizer(min_df=1,ngram_range=(1,2))
Count_train=Count.fit_transform(X_train)
Count_test=Count.transform(X_test)

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

In [20]:
steps=[('clf',MultinomialNB())]
pipeline=Pipeline(steps)

In [21]:
# Parameter alpha is a smoothing parameter in Naive Bayes classifier for multinomial models

In [22]:
param_grid={'clf__alpha':np.linspace(0,10,50)}

In [23]:
# Using Gridsearch to find the optimal parameter alpha

In [24]:
from sklearn.model_selection import GridSearchCV

In [25]:
clf_cv=GridSearchCV(pipeline,param_grid)

In [26]:
clf_cv.fit(Count_train,y_train)

C:\Users\HoT\Anaconda3_2\lib\site-packages\sklearn\naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
C:\Users\HoT\Anaconda3_2\lib\site-packages\sklearn\naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
C:\Users\HoT\Anaconda3_2\lib\site-packages\sklearn\naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__alpha': array([  0.     ,   0.20408,   0.40816,   0.61224,   0.81633,   1.02041,
         1.22449,   1.42857,   1.63265,   1.83673,   2.04082,   2.2449 ,
         2.44898,   2.65306,   2.85714,   3.06122,   3.26531,   3.46939,
         3.67347,   3.87755,   4.08163,   4.28571,   4....      8.57143,   8.77551,   8.97959,   9.18367,   9.38776,   9.59184,
         9.79592,  10.     ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [27]:
print("The best parameter is",clf_cv.best_params_)

The best parameter is {'clf__alpha': 3.8775510204081636}


In [28]:
print('Accuracy for the sub-train set = {}'.format(clf_cv.score(Count_train,y_train)))
print('Accuracy for the sub-test set = {}'.format(clf_cv.score(Count_test,y_test)))

Accuracy for the sub-train set = 0.9996789727126806
Accuracy for the sub-test set = 0.993581514762516


In [29]:
# Loading the test set for tagging prediction

In [30]:
df_test=pd.read_csv("offsite-tagging-test-set.csv",index_col="id")

In [31]:
df_test.head()

,text
id,
6,南華添鋒力 簽前厄瓜多爾國腳保耶 港超勁旅南華宣布羅致前厄瓜多爾國腳菲力斯保耶（Felix ...
128,如果大學$0捐款 科大嶺南將蝕過千萬元 據now新聞台報道，身兼8大校監的特首梁振英曾以大學...
136,英超最強火力對碰 雙城爭冠靠鋒霸 英超今季風起雲湧，傳統「Big 5」只剩兩隊名列積分榜前5...
213,【01球評】膺半程冠軍 阿仙奴今季不奪標更待何時？ 近年「兵工廠」每季的起落都離不開一個循環...
658,【書商失蹤】梁振英：希望失蹤的李波本人提供資料 行政長官梁振英出席行政會議前見記者，被問及李...


In [32]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 974 entries, 6 to 94324
Data columns (total 1 columns):
text    974 non-null object
dtypes: object(1)
memory usage: 15.2+ KB


In [33]:
# Preprocessing the text data

In [34]:
df_test['Clean']=df_test['text'].str.findall('[^\da-zA-Z\s\W_%s]'%Stopword)

In [35]:
row_t,col_t=df_test.shape

In [36]:
for i in range(row_t):
    df_test.iloc[i,1]=''.join(df_test.iloc[i,1])
    df_test.iloc[i,1]=' '.join(jieba.cut(df_test.iloc[i,1],cut_all=False))

In [37]:
df_test.head()

,text,Clean
id,,
6,南華添鋒力 簽前厄瓜多爾國腳保耶 港超勁旅南華宣布羅致前厄瓜多爾國腳菲力斯保耶（Felix ...,南華 添鋒力 簽前 厄瓜 多爾國腳 保耶港 超勁 旅南華 宣布 羅 致前 厄瓜 多爾國 腳菲...
128,如果大學$0捐款 科大嶺南將蝕過千萬元 據now新聞台報道，身兼8大校監的特首梁振英曾以大學...,大學 捐款 科大 嶺南 將蝕 過千萬元據 新聞台 報道 身兼 大校 監 特首 梁振英 曾大學...
136,英超最強火力對碰 雙城爭冠靠鋒霸 英超今季風起雲湧，傳統「Big 5」只剩兩隊名列積分榜前5...,英超 最強 火力 對 碰 雙城爭 冠靠鋒 霸 英超 今季 風起 雲湧 傳統 剩 兩隊 名列 ...
213,【01球評】膺半程冠軍 阿仙奴今季不奪標更待何時？ 近年「兵工廠」每季的起落都離不開一個循環...,球 評膺 半程 冠 軍 阿仙奴 今季 不奪標 更 待 何時 近年 兵工 廠 每季 起落 離不...
658,【書商失蹤】梁振英：希望失蹤的李波本人提供資料 行政長官梁振英出席行政會議前見記者，被問及李...,書商 失 蹤 梁 振 英 希望 失 蹤 李 波 本人 提供 資料 行政 長 官 梁振英 出席...


In [38]:
X_rtest=df_test.iloc[:,1].values

In [39]:
Ctest=Count.transform(X_rtest)

In [40]:
pred=clf_cv.predict(Ctest)

# The resulting tagging prediction for the given test set

In [41]:
pred

array(['足球', '梁振英', '足球', '足球', '梁振英', '梁振英', '梁振英', '足球', '足球', '足球',
       '足球', '足球', '足球', '足球', '足球', '足球', '足球', '梁振英', '梁振英', '梁振英',
       '美國大選', '梁振英', '梁振英', '梁振英', '梁振英', '梁振英', '足球', '梁振英', '足球', '梁振英',
       '足球', '梁振英', '足球', '足球', '足球', '足球', '梁振英', '美國大選', '足球', '美國大選',
       '足球', '梁振英', '足球', '足球', '足球', '足球', '足球', '足球', '足球', '足球', '足球',
       '美國大選', '足球', '足球', '美國大選', '足球', '足球', '足球', '足球', '足球', '足球',
       '美國大選', '美國大選', '美國大選', '美國大選', '足球', '足球', '美國大選', '足球', '足球',
       '足球', '足球', '足球', '梁振英', '足球', '足球', '足球', '足球', '足球', '足球', '足球',
       '美國大選', '梁振英', '美國大選', '美國大選', '美國大選', '足球', '梁振英', '足球', '足球',
       '足球', '美國大選', '梁振英', '足球', '足球', '足球', '足球', '足球', '足球', '足球',
       '美國大選', '足球', '足球', '足球', '足球', '梁振英', '足球', '足球', '足球', '梁振英',
       '美國大選', '梁振英', '足球', '足球', '足球', '梁振英', '梁振英', '美國大選', '美國大選',
       '美國大選', '美國大選', '足球', '足球', '美國大選', '梁振英', '美國大選', '梁振英', '美國大選',
       '足球', '足球', '足球', '足球', '梁振英', '足球', '美國大選', '足球', '足球', '